# smelli analysis of WCxf files

## Loading smelli etc.

In [1]:
import os
import time
from smelli import GlobalLikelihood

### Global Likelihood

List of sub-likelihoods to be included in the global likelihood. By default (`None`), all available likelihoods are included.

In [2]:
likelihoods_included = None # [
    # 'fast_likelihood_quarks_fixckm.yaml' # use only one of the fist two likelihoods depending on your CKM choice
    # ,'fast_likelihood_quarks.yaml' # use only one of the fist two likelihoods depending on your CKM choice
    # ,'fast_likelihood_leptons.yaml'
    # ,'likelihood_ewpt.yaml'
    # ,'likelihood_eeww.yaml'
    # ,'likelihood_lept.yaml'
    # ,'likelihood_rd_rds.yaml'
    # ,'likelihood_lfu_fccc.yaml'
    # ,'likelihood_lfu_fcnc.yaml'
    # ,'likelihood_bcpv.yaml'
    # ,'likelihood_bqnunu.yaml'
    # ,'likelihood_lfv.yaml'
    # ,'likelihood_zlfv.yaml'
    # ,'likelihood_higgs.yaml'
#]

Creating a global likelihood instance

In [3]:
gl = GlobalLikelihood(
    eft='SMEFT', 
    basis='Warsaw', 
    fix_ckm=True, # determines whether the CKM is fixed to its SM value (True) or extracted in the presence of NP (False).
    include_likelihoods=likelihoods_included
)

## Loop over WCxf files

Specify the name of the directory containing all WCxf files for this analysis. By default, this directy should be placed in `../matching/WCxf`.

In [4]:
model_name = 'Majorana_e_scan5'

Set directory for WCxf files

In [5]:
directory = os.path.join(
    os.path.abspath(os.path.join(os.getcwd(), os.pardir)), 
    'matching', 
    'WCxf', 
    model_name
)
directory

'/Users/fwilsch/Repositories/flavored-DM/flavor/matching/WCxf/Majorana_e_scan5'

### Read in WCxf files and generate parameter points

In [6]:
# this is the list of all global likelihood parameter points
param_points = list()

for filename in sorted(os.listdir(directory), key=lambda x:(len(x),x[1:])):
    f = os.path.join(directory, filename)
    # checking if it is a file
    if os.path.isfile(f):
        # print(f)
        try:
            param_points.append(gl.parameter_point(f))
        except:
            print('Could not read: ' + str(f)) # print files that could not be read by smelli

print('Number of WCxf files read: ' + str(len(param_points)))

Number of WCxf files read: 6702


Uncomment the line below to test only the first 10 parameter points.

In [ ]:
# param_points = param_points[:10] # test only on the first 10 parameter points

## Compute log-likelihood ratio and observables table for all parameter points

In [8]:
obstable_list = []
point_list = []
point_list_fix = []
output_list = {}
error_list = []
counter = 0

failed_counter = 0
passed_counter = 0
errors_counter = 0


# compute likelihood and observable table for each parameter point
point_list = param_points

for point in point_list:
    counter += 1
    try:
        tmpL = point.log_likelihood_dict()
        tmpO = point.obstable().sort_values('pull SM', ascending=False)
        obstable_list.append(tmpO)
        output_list.update({counter : {'log_likelihood_dict' : tmpL , 'obstable' : tmpO}})
        if tmpL['global'] > -2:
            passed_counter += 1
        else:
            failed_counter += 1
    except:
        errors_counter += 1
        error_list.append(counter-1)
        print(counter-1) # prints the number id of the file for which smelli crashed


# print number of points that pass and fail
print("errors: \t", errors_counter)
print("points passed: \t", passed_counter)
print("points failed: \t", failed_counter)

print(error_list)

output_list.update({'overview' : {'passed' : passed_counter , 'failed' : failed_counter , 'error' : errors_counter}})

errors: 	 0
points passed: 	 7
points failed: 	 3
[]


### Create a summary of the most relevant observables

In [9]:
# determine observables with pull >2sigma w.r.t. to SM
dict = {}

for index, obs in obstable_list[0].iterrows():
    dict.update({index: 0})

for obstable_entry in obstable_list:
    for index, obs in obstable_entry.iterrows():
        if obs['pull SM'] >= 2:
            dict[index] += 1
        
dict = {x:y for x,y in dict.items() if y!=0}

output_list.update({'summary' : dict})

dict

{'BR(mu->eee)': 3, 'BR(mu->egamma)': 3, 'CR(mu->e, Au)': 1, 'CR(mu->e, Ti)': 1}

### Check some examples

In [10]:
output_list[1]['log_likelihood_dict']

{'fast_likelihood_quarks_fixckm.yaml': -5.606905915556126e-06,
 'fast_likelihood_leptons.yaml': 0.032802174906152004,
 'likelihood_ewpt.yaml': -0.00034179804948308856,
 'likelihood_eeww.yaml': -6.719187165771245e-06,
 'likelihood_lept.yaml': 1.4526613245635644e-09,
 'likelihood_rd_rds.yaml': -2.7623591947190107e-07,
 'likelihood_lfu_fccc.yaml': 8.768985537699336e-10,
 'likelihood_lfu_fcnc.yaml': -1.901412360894028e-11,
 'likelihood_bcpv.yaml': 4.770406292209373e-12,
 'likelihood_bqnunu.yaml': -1.820154693632503e-09,
 'likelihood_lfv.yaml': -3.473226087180592e-05,
 'likelihood_zlfv.yaml': -5.31947819126799e-11,
 'likelihood_higgs.yaml': -2.285259073531165e-05,
 'global': 0.032390190118027684}

In [11]:
output_list[1]['obstable']

,th. unc.,experiment,exp. unc.,theory,pull exp.,pull SM
R_mu,0,20.7842,0.0335,20.735603,1.450652,0.053018
A(Z->ee),0,0.151321,0.00193,0.147656,1.898453,0.025009
AFB(Z->tautau),0,0.0188,0.0017,0.016352,1.440273,0.023248
R_e,0,20.8038,0.0497,20.735627,1.371699,0.022403
a_e,0.0,0.00116,0.0,0.00116,1.938208,0.019037
...,...,...,...,...,...,...
m_W,0,80.379511,0.012102,80.355071,2.019562,-0.021516
AFB(Z->bb),0,0.0992,0.0016,0.103517,2.698333,-0.027302
R_tau,0,20.7644,0.0448,20.782615,0.406589,-0.029977
A(Z->tautau),0,0.1433,0.004134,0.147654,1.053329,-0.036143


In [12]:
output_list['summary']

{'BR(mu->eee)': 3, 'BR(mu->egamma)': 3, 'CR(mu->e, Au)': 1, 'CR(mu->e, Ti)': 1}

## Exporting results

In [13]:
import pickle

### Export points

In [ ]:
os.makedirs(os.path.join('results', model_name), exist_ok=True)

with open(os.path.join(os.path.join('results', model_name), 'results_scan.dat'), "wb") as outfile: 
    pickle.dump(output_list, outfile)

# exporting likelihood points
with open(os.path.join(os.path.join('results', model_name), 'likelihhod_points.dat'), "wb") as f:
    pickle.dump(point_list, f)

# exporting obstables
with open(os.path.join(os.path.join('results', model_name), 'obstables_points.dat'), "wb") as f:
    pickle.dump(obstable_list, f)

### Import points

In [ ]:
# with open(os.path.join(model_name, 'likelihhod_points.dat'), "rb") as f:
#     loaded_points = pickle.load(f)

# with open(os.path.join(model_name, 'obstables_points.dat'), "rb") as f:
#     loaded_obstables = pickle.load(f)

# with open(os.path.join(model_name, 'results_scan.dat'), "rb") as f:
#     loaded_output = pickle.load(f)